In [ ]:
from google3.learning.deepmind.xmanager2.client import xmanager_api
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections

xm_client = xmanager_api.XManagerApi(xm_deployment_env='alphabet')
MEDIUM_SIZE = 14
mpl.rcParams.update({
    'font.size': MEDIUM_SIZE,
    'axes.labelsize': MEDIUM_SIZE,
    'axes.titlesize': MEDIUM_SIZE,
})
mpl.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 20
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.edgecolor'] = '#777777'
plt.rcParams['axes.facecolor'] = '#FFFFFF'

SMALL_SIZE = 16
MEDIUM_SIZE = 15
BIGGER_SIZE = 20

plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)  # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

elegant_palette = sns.color_palette('muted')

In [ ]:
def read_xm_metrics(example_xid, metric_name, unit_id, lowest=True):
  experiment = xm_client.get_experiment(example_xid)
  work_unit = experiment.get_work_unit(unit_id)
  all_series = work_unit.list_measurement_series()
  # Read measurement series metadata.
  for series in all_series:
    if series.label == metric_name:
      # Read measurement points data.
      all_measurements = []
      for measurement in series.measurements:
        all_measurements.append(measurement.objective_value)
      if lowest:
        return min(all_measurements)
      else:
        return all_measurements

## lsm_300min_mood_vs_activity

In [ ]:
task_name = 'lsm_300min_mood_vs_activity'

metrics = [
    f'linear_eval/{task_name}_accuracy',
    f'linear_eval/{task_name}_loss',
]

xm_id = 121775751
experiment = xm_client.get_experiment(xm_id)
num_of_units = experiment.get_num_work_units()
xm_exp_dict = collections.defaultdict(list)

for id in range(num_of_units):
  real_id = id + 1
  work_unit = experiment.get_work_unit(real_id)
  key_list = work_unit.parameters.keys()
  xm_exp_dict['unit_id'].append(id)
  xm_exp_dict['xm_id'].append(xm_id)
  for param_name in key_list:
    xm_exp_dict[param_name].append(work_unit.parameters[param_name])
  for metric in metrics:
    xm_exp_dict[metric].append(
        read_xm_metrics(xm_id, metric, real_id, lowest=False)
    )

df = pd.DataFrame(xm_exp_dict)
df.info(verbose=True)

# Calculating the highest accuracy and lowest loss
df['max_accuracy'] = df[f'linear_eval/{task_name}_accuracy'].apply(max)
df['min_loss'] = df[f'linear_eval/{task_name}_loss'].apply(min)

# Ranking by highest accuracy and lowest loss
df_sorted_by_accuracy = df.sort_values(by='max_accuracy', ascending=False)
df_sorted_by_loss = df.sort_values(by='min_loss', ascending=True)

# Display the sorted DataFrames
print('DataFrame sorted by highest accuracy:')
print(df_sorted_by_accuracy)

print('\nDataFrame sorted by lowest loss:')

In [ ]:
# Get unique learning rates
unique_metric = f'config.linear_probe.{task_name}.lr_configs.base_learning_rate'
abalation_metric = (
    f'config.linear_probe.{task_name}.lr_configs.total_steps'
)
weight_decay_metric = f'config.linear_probe.{task_name}.weight_decay'
unique_lr = df[unique_metric].unique()
print('unique_lr: ', unique_lr)
# Set an elegant color palette
palette = sns.color_palette('Set2')
color_mapping = dict(zip(df[abalation_metric].unique(), palette))
df = df[df[weight_decay_metric] == 1e-5]
# Create subplots
plt.figure(figsize=(6 * len(unique_lr), 6 * len(metrics)), dpi=600)

for row_idx, metric_name in enumerate(metrics):
  for i, lr in enumerate(unique_lr):
    plt.subplot(len(metrics), len(unique_lr), row_idx * len(unique_lr) + i + 1)
    subset = df[df[unique_metric] == lr]
    for _, row in subset.iterrows():
      x = list(range(len(row[metric_name])))
      y = row[metric_name]
      weight_decay = row[abalation_metric]
      plt.plot(x, y, label=f'{weight_decay}', color=color_mapping[weight_decay])
    plt.title(f'LR: {lr}')
    if row_idx == len(metrics) - 1:
      plt.xlabel('Pretraining Steps (K)')
    else:
      plt.xlabel('')
    if i == 0:
      plt.ylabel(metric_name)
    else:
      plt.ylabel('')
    # if row_idx == 0 and i == len(unique_lr) - 1:
    plt.legend(title=abalation_metric.split('.')[-1], frameon=False)

plt.tight_layout()
plt.suptitle(
    f"Ablation Study of {task_name} in  {unique_metric.split('.')[-1]} and"
    f" {abalation_metric.split('.')[-1]} Across Metrics + 1M Dataset +"
    ' Small ViT',
    y=1.02,
)
plt.show()

## lsm_300min_600_activities_balanced

In [ ]:
task_name = 'lsm_300min_600_activities_balanced'
metrics = [
    f'linear_eval/{task_name}_accuracy',
    f'linear_eval/{task_name}_loss',
]

xm_id = 121792637
experiment = xm_client.get_experiment(xm_id)
num_of_units = experiment.get_num_work_units()
xm_exp_dict = collections.defaultdict(list)

for id in range(num_of_units):
  real_id = id + 1
  work_unit = experiment.get_work_unit(real_id)
  key_list = work_unit.parameters.keys()
  xm_exp_dict['unit_id'].append(id)
  xm_exp_dict['xm_id'].append(xm_id)
  for param_name in key_list:
    xm_exp_dict[param_name].append(work_unit.parameters[param_name])
  for metric in metrics:
    xm_exp_dict[metric].append(read_xm_metrics(xm_id, metric, real_id, lowest=False))

df = pd.DataFrame(xm_exp_dict)
df.info(verbose=True)

# Calculating the highest accuracy and lowest loss
df['max_accuracy'] = df[f'linear_eval/{task_name}_accuracy'].apply(max)
df['min_loss'] = df[f'linear_eval/{task_name}_loss'].apply(min)

# Ranking by highest accuracy and lowest loss
df_sorted_by_accuracy = df.sort_values(by='max_accuracy', ascending=False)
df_sorted_by_loss = df.sort_values(by='min_loss', ascending=True)

# Display the sorted DataFrames
print("DataFrame sorted by highest accuracy:")
print(df_sorted_by_accuracy)

print("\nDataFrame sorted by lowest loss:")


In [ ]:
# Get unique learning rates
unique_metric = f'config.linear_probe.{task_name}.lr_configs.base_learning_rate'
abalation_metric = (
    f'config.linear_probe.{task_name}.lr_configs.total_steps'
)
weight_decay_metric = f'config.linear_probe.{task_name}.weight_decay'
unique_lr = df[unique_metric].unique()
print('unique_lr: ', unique_lr)
# Set an elegant color palette
palette = sns.color_palette('Set2')
color_mapping = dict(zip(df[abalation_metric].unique(), palette))
df = df[df[weight_decay_metric] == 1e-5]
# Create subplots
plt.figure(figsize=(6 * len(unique_lr), 6 * len(metrics)), dpi=600)

for row_idx, metric_name in enumerate(metrics):
  for i, lr in enumerate(unique_lr):
    plt.subplot(len(metrics), len(unique_lr), row_idx * len(unique_lr) + i + 1)
    subset = df[df[unique_metric] == lr]
    for _, row in subset.iterrows():
      x = list(range(len(row[metric_name])))
      y = row[metric_name]
      weight_decay = row[abalation_metric]
      plt.plot(x, y, label=f'{weight_decay}', color=color_mapping[weight_decay])
    plt.title(f'LR: {lr}')
    if row_idx == len(metrics) - 1:
      plt.xlabel('Pretraining Steps (K)')
    else:
      plt.xlabel('')
    if i == 0:
      plt.ylabel(metric_name)
    else:
      plt.ylabel('')
    # if row_idx == 0 and i == len(unique_lr) - 1:
    plt.legend(title=abalation_metric.split('.')[-1], frameon=False)

plt.tight_layout()
plt.suptitle(
    f"Ablation Study of {task_name} in  {unique_metric.split('.')[-1]} and"
    f" {abalation_metric.split('.')[-1]} Across Metrics + 1M Dataset +"
    ' Small ViT',
    y=1.02,
)
plt.show()

## lsm_300min_2000_mood_balanced

In [ ]:
task_name = 'lsm_300min_2000_mood_balanced'
metrics = [
    f'linear_eval/{task_name}_accuracy',
    f'linear_eval/{task_name}_loss',
]

xm_id = 121798270
experiment = xm_client.get_experiment(xm_id)
num_of_units = experiment.get_num_work_units()
xm_exp_dict = collections.defaultdict(list)

for id in range(num_of_units):
  real_id = id + 1
  work_unit = experiment.get_work_unit(real_id)
  key_list = work_unit.parameters.keys()
  xm_exp_dict['unit_id'].append(id)
  xm_exp_dict['xm_id'].append(xm_id)
  for param_name in key_list:
    xm_exp_dict[param_name].append(work_unit.parameters[param_name])
  for metric in metrics:
    xm_exp_dict[metric].append(read_xm_metrics(xm_id, metric, real_id, lowest=False))

df = pd.DataFrame(xm_exp_dict)
df.info(verbose=True)

# Calculating the highest accuracy and lowest loss
df['max_accuracy'] = df[f'linear_eval/{task_name}_accuracy'].apply(max)
df['min_loss'] = df[f'linear_eval/{task_name}_loss'].apply(min)

# Ranking by highest accuracy and lowest loss
df_sorted_by_accuracy = df.sort_values(by='max_accuracy', ascending=False)
df_sorted_by_loss = df.sort_values(by='min_loss', ascending=True)

# Display the sorted DataFrames
print("DataFrame sorted by highest accuracy:")
print(df_sorted_by_accuracy)

print("\nDataFrame sorted by lowest loss:")


In [ ]:
# Get unique learning rates
unique_metric = f'config.linear_probe.{task_name}.lr_configs.base_learning_rate'
abalation_metric = (
    f'config.linear_probe.{task_name}.lr_configs.total_steps'
)
weight_decay_metric = f'config.linear_probe.{task_name}.weight_decay'
unique_lr = df[unique_metric].unique()
print('unique_lr: ', unique_lr)
# Set an elegant color palette
palette = sns.color_palette('Set2')
color_mapping = dict(zip(df[abalation_metric].unique(), palette))
df = df[df[weight_decay_metric] == 1e-5]
# Create subplots
plt.figure(figsize=(6 * len(unique_lr), 6 * len(metrics)), dpi=600)

for row_idx, metric_name in enumerate(metrics):
  for i, lr in enumerate(unique_lr):
    plt.subplot(len(metrics), len(unique_lr), row_idx * len(unique_lr) + i + 1)
    subset = df[df[unique_metric] == lr]
    for _, row in subset.iterrows():
      x = list(range(len(row[metric_name])))
      y = row[metric_name]
      weight_decay = row[abalation_metric]
      plt.plot(x, y, label=f'{weight_decay}', color=color_mapping[weight_decay])
    plt.title(f'LR: {lr}')
    if row_idx == len(metrics) - 1:
      plt.xlabel('Pretraining Steps (K)')
    else:
      plt.xlabel('')
    if i == 0:
      plt.ylabel(metric_name)
    else:
      plt.ylabel('')
    # if row_idx == 0 and i == len(unique_lr) - 1:
    plt.legend(title=abalation_metric.split('.')[-1], frameon=False)

plt.tight_layout()
plt.suptitle(
    f"Ablation Study of {task_name} in  {unique_metric.split('.')[-1]} and"
    f" {abalation_metric.split('.')[-1]} Across Metrics + 1M Dataset +"
    ' Small ViT',
    y=1.02,
)
plt.show()